In [2]:
from google.colab import drive
drive.mount('/content/drive')
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import re
import numpy as np
import os
import pandas

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Using TensorFlow backend.


In [0]:
# PARAMETERS CAN BE CHANGED HERE WITHOUT MESSING WITH THE CODE
kmersize=5
modelout=4**kmersize

In [0]:
"""
this function provides information about the array    
"""
def array_inspect(x):
    print ("shape=",(x.shape),"len=",len(x),"ndim=",x.ndim,"size=",x.size,"dtype=",x.dtype)

In [5]:
eventalign=np.loadtxt("/content/drive/My Drive/Colab Notebooks/tensorflow_basecaller/datasets/reads-ref.eventalign.txt", delimiter = ' ')

ValueError: ignored

In [0]:
print(eventalign)

In [0]:
model=tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu)) # uses 128 neurons and is a feed forward rectilinear relu
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(1024, activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(0.2))
#model.add(tf.keras.layers.Dense(512, activation=tf.nn.relu)) # do the same thig for hidden layer 2
#model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(modelout, activation=tf.nn.softmax)) # output layer with number of classifications (256) use softmax for output distribution

In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [0]:
checkpoint_path = "/content/drive/My Drive/Colab Notebooks/tensorflow_basecaller/training_7mer/jochum/jochum_cp1.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='loss', 
                                      verbose=1, 
                                      save_weights_only=True, 
                                      save_best_only=True,
                                      period = 10,
                                      mode='min')
callbacks_list = [checkpoint]
#model.load_weights(checkpoint_path)

In [1]:
x_val = x_train[0:800000]
partial_x_train = x_train[800000:]
y_val = y_train[0:800000]
partial_y_train = y_train[800000:]

history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=1000,
                    validation_data=(x_val, y_val),
                    batch_size=30000,
                    verbose=1,
                    callbacks = callbacks_list)
model.summary()

NameError: ignored

In [0]:
results = model.evaluate(x_train, y_train)
print(results)

In [0]:
min_pred = 0
max_pred = min_pred+100
preds = model.predict(x_train[min_pred:max_pred])
concat_reads = ''
concat_ref = ''

for arr in preds:
  concat_reads += str(dict2[np.argmax(arr)])

for arr in y_train[min_pred:max_pred]:
  concat_ref += str(dict2[arr])

read_len = len(concat_reads)
accurate = 0

for i in range(0,read_len):
  if concat_reads[i] == concat_ref[i]:
    accurate+=1

ret_arr = [concat_reads, concat_ref]
print(ret_arr)

In [0]:
history_dict = history.history
history_dict.keys()
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
plt.clf()   # clear figure
acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()